# Find auxtel slew and block times

In [6]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, HTML

from astropy.time import Time, TimeDelta

from rubin_scheduler.site_models import Almanac
from rubin_scheduler.utils import Site

from lsst_efd_client import EfdClient
import requests
import urllib

In [133]:
# On the RSP, access token available from lsst.rsp
try:
    from lsst.rsp import get_access_token
# But otherwise ..
except ImportError:
    def get_access_token(token_file : str | None = None) -> str:
        if token_file is not None:
            with open(token_file, "r") as f:
                token = f.read()
                token = token.rstrip("\n")
        else:
            token = os.environ.get("ACCESS_TOKEN")
        if token is None:
            warnings.warn("No RSP token available.")
        return token

def get_clients(token_file : str | None = None) -> dict:
    """Return site-specific client connections. 

    Parameters
    ----------
    token_file : `str` or None
        File containing an RSP token.

    Returns
    -------
    endpoints : `dict`
        Dictionary with `efd`, `obsenv`, 
        `narrative_log`, and `exposure_log`
        connection information.
        For the obsenv, narrative log and exposure log, these are only
        defined for the summit or USDF.

    Note
    ----
    The authentication token required to access the log services
    is an RSP token, and is RSP site-specific. 
    For users outside the RSP, a token can be created as described in
    https://nb.lsst.io/environment/tokens.html
    """
    # Set up authentication
    token = get_access_token(token_file)
    auth = ("user", token)
    # This authentication is for nightlog, exposurelog, nightreport currently
    # But I think it's the same underlying info for EfdClient i.e.
    # https://github.com/lsst/schedview/blob/e11fbd51ee5e22d11fef9a52f66dfcc082181cb6/schedview/app/scheduler_dashboard/influxdb_client.py
    # For lots more information on rubin tokens see DMTN-234.
    # For information on scopes, see DMTN-235.
    
    # let's do this like lsst.summit.utils.getSite but simpler
    site = "UNKNOWN"
    location = os.getenv("EXTERNAL_INSTANCE_URL", "")
    if "tucson-teststand" in location:
        site = "tucson"
    elif "summit-lsp" in location:
        site = "summit"
    elif "base-lsp" in location:
        site = "base"
    elif "usdf-rsp" in location:
        site = "usdf"
    # If location not set, next step is to check hostname
    elif location == "":
        hostname = os.getenv("HOSTNAME", "")
        interactiveNodes = ("sdfrome", "sdfiana")
        if hostname.startswith(interactiveNodes):
            site = "usdf"
        elif hostname == "htcondor.ls.lsst.org":
            site = "base"
        elif hostname == "htcondor.cp.lsst.org":
            site = "summit"

    
    if site == "summit":
        api_base = "https://summit-lsp.lsst.codes"
        efd_client = EfdClient("summit_efd")
        obsenv_client = EfdClient("summit_efd", db_name="lsst.obsenv")
    elif site == "tucson":
        api_base = None
        efd_client = EfdClient("tucson_teststand_efd")
        obsenv_client = EfdClient("tucson_teststand_efd", db_name="lsst.obsenv")
    elif site == "base":
        api_base = "https://base-lsp.slac.lsst.codes"
        efd_client = EfdClient("base_efd")
        obsenv_client = EfdClient("base_efd", db_name="lsst.obsenv")
    elif site == "usdf":
        # For tokens, need to distinguish between dev and prod
        if "dev" in location:
            api_base = "https://usdf-rsp-dev.slac.stanford.edu"
        else:
            api_base = "https://usdf-rsp.slac.stanford.edu"
        efd_client = EfdClient("usdf_efd")
        obsenv_client = EfdClient("usdf_efd", db_name='lsst.obsenv')
        # Also set up some env variables.
        os.environ["no_proxy"] += ",.consdb"
        os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/shared/rubin_sim_data"
    else:
        # Assume USDF prod (for any UNKNOWN)
        efd_client = EfdClient("usdf_efd")
        obsenv_client = EfdClient("usdf_efd", db_name='lsst.obsenv')
        api_base = "https://usdf-rsp.slac.stanford.edu"
    
    narrative_log =  "/narrativelog/messages"
    exposure_log = "/exposurelog/messages"
    nightreport =  "/nightreport/reports"
    consdb_query = "/consdb/query"
    
    endpoints = {'api_base': api_base, 'auth': auth, 
                'efd': efd_client, 'obsenv': obsenv_client, 
                'consdb': consdb_query,
                'narrative_log': narrative_log, 'exposure_log': exposure_log,  
                'nightreport': nightreport}
    
    # If some verbose output is desired
    # We'll put this here to make it easier to avoid printing the auth token
    endpoints_string = f"base url: {endpoints['api_base']} " 
    endpoints_string += f"efd host: {endpoints['efd'].influx_client.host}"
    endpoints['string'] = endpoints_string

    return endpoints


def query_logging_services(endpoint: str, auth: tuple, params: dict) -> pd.DataFrame:
    """Send `get` query to logging services. 
    
    Parameters
    ----------
    endpoint : `str`
        The URL to send the query to.
        Usually like `https://usdf-rsp.slac.stanford.edu/narrativelog/messages`
    auth : `tuple`
        The username and password for authentication.
        The username can be any string, the password should be an RSP token.
        See e.g. https://nb.lsst.io/environment/tokens.html 
    params : `dict`
        Dictionary of parameters for the REST API query.
        See docs for each service for more details.

    Returns
    -------
    messages : `pd.DataFrame`
        The returned log messages (if any available), in a dataframe.
    """
    # Very often, requests from the logging endpoints fail the first time.
    response = requests.get(endpoint, auth=auth, params=params)
    # Try twice.
    if response.status_code != 200:
        response = requests.get(endpoint, auth=auth, params=params)
    if response.status_code != 200:
        err_string = f"{endpoint} "
        err_string += " unavailable."
        print(response)
        print(err_string)
        messages = []
    else:
        messages = response.json()
    messages = pd.DataFrame(messages)
    return messages

def query_consdb(endpoint: str, auth: tuple, params: dict) -> pd.DataFrame:
    """Send `post` query to consdb.
    
    Parameters
    ----------
    endpoint : `str`
        The URL to send the query to.
        Usually like `https://usdf-rsp.slac.stanford.edu/consdb/query`
    auth : `tuple`
        The username and password for authentication.
        The username can be any string, the password should be an RSP token.
        See e.g. https://nb.lsst.io/environment/tokens.html 
    params : `dict`
        Dictionary of parameters for the REST API query.
        See docs for each service for more details.

    Returns
    -------
    messages : `pd.DataFrame`
        The returned visits information (if any available), in a dataframe.
    """
    # Very often, requests from the logging endpoints fail the first time.
    response = requests.post(endpoint, auth=auth, json=params)
    # Try twice.
    if response.status_code != 200:
        response = requests.post(endpoint, auth=auth, json=params)
    if response.status_code != 200:
        err_string = f"{endpoint} "
        err_string += " unavailable."
        print(response)
        print(err_string)
        messages = []
    else:
        messages = response.json()
    messages = pd.DataFrame(messages['data'], columns=messages['columns'])
    return messages

In [134]:
endpoints = get_clients(token_file = '/Users/lynnej/.lsst/rsp_prod')
endpoints['string'], endpoints.keys()

('base url: https://usdf-rsp.slac.stanford.edu efd host: usdf-rsp.slac.stanford.edu',
 dict_keys(['api_base', 'auth', 'efd', 'obsenv', 'consdb', 'narrative_log', 'exposure_log', 'nightreport', 'string']))

In [135]:
# What night do you want to simulate? 
DAYOBS = '2025-02-25'
day_obs_mjd = int(Time(DAYOBS).mjd)
day_obs_int = int(DAYOBS.replace('-', ''))
site = Site('LSST')
almanac = Almanac()
night_events = almanac.get_sunset_info(evening_date=DAYOBS, longitude=site.longitude_rad)
civil_sunset = Time(night_events['sunset'], format='mjd', scale='utc') 
sunset = Time(night_events['sun_n12_setting'], format='mjd', scale='utc') 
sunrise = Time(night_events['sun_n12_rising'], format='mjd', scale='utc')
survey_length = sunrise.mjd - sunset.mjd
night_length = sunrise.mjd - sunset.mjd
print(sunset, sunrise, night_length*24)

60732.00944711408 60732.40171818761 9.414505764842033


In [287]:
def get_visits(instrument : str, day_obs_int_min : int, day_obs_int_max : int, consdb : str, auth : tuple) -> pd.DataFrame :
    
    quicklook_cols = ['sky_bg_median', 'zero_point_median', 'psf_sigma_median', 'psf_area_median', 'seeing_zenith_500nm_median', 'eff_time_median']
    quicklook = ', '.join([f'q.{col}' for col in quicklook_cols])
    
    # Querying separately and joining in pandas works 
    visit_query = f'''
        SELECT * 
        FROM cdb_{instrument}.visit1
         WHERE day_obs >= {day_obs_int_min}
         and day_obs  <= {day_obs_int_max}
    '''
    visit_query = " ".join(visit_query.split())

    # Quicklook not present yet for latiss
    quicklook_query = f'''
        SELECT q.*  FROM cdb_{instrument}.visit1_quicklook as q,
        cdb_{instrument}.visit1 as v
         WHERE q.visit_id = v.visit_id and 
         v.day_obs >= {day_obs_int_min} 
         and v.day_obs <= {day_obs_int_max}
    '''
    quicklook_query = " ".join(quicklook_query.split())
    
    visits = query_consdb(consdb, auth, params={'query': visit_query})
    if len(visits) > 0:
        visits.set_index('visit_id', inplace=True)
        display(Markdown(f"Retrieved {len(visits)} visits from consdb"))
    
    quicklook = query_consdb(consdb, auth, params={'query': quicklook_query})
    
    if len(quicklook) > 0:
        quicklook.set_index('visit_id', inplace=True)
        visits = visits.join(quicklook, lsuffix='', rsuffix='_q')
        display(Markdown(f"And added quicklook stats"))
    
    if len(visits) == 0:
        display(Markdown(f"No visits for {instrument} between {day_obs_int_min} to {day_obs_int_max} retrieved from consdb"))
        return visits

    
    visits.sort_values(by='exp_midpt_mjd', inplace=True)
    # Add time between visits
    prev_visit_start = np.concatenate([np.array([0]), visits.obs_start_mjd[0:-1]])
    prev_visit_end = np.concatenate([np.array([0]), visits.obs_end_mjd[0:-1]])
    delta_t = np.concatenate([np.array([0]), (visits.obs_start_mjd[1:].values - visits.obs_end_mjd[:-1].values) * 24 * 60 * 60])
    visits['prev_obs_start_mjd'] = prev_visit_start
    visits['prev_obs_end_mjd'] = prev_visit_end
    visits['visit_gap'] = delta_t

    # Remove when bug in consdb fixed 
    visits['alt'] =  90 - visits['altitude']
    
    return visits

In [288]:
visits = get_visits('latiss', day_obs_int, day_obs_int, endpoints['api_base'] + endpoints['consdb'],
                    endpoints['auth'])
# Drop non-science visits 
visits = visits.query("exp_time >= 5 and img_type not in ('DARK', 'FLAT', 'CWFS')")

Retrieved 637 visits from consdb

In [290]:
cols = ['seq_num', 'physical_filter', 's_ra', 's_dec', 'altitude', 'azimuth', 'sky_rotation', 'obs_start', 'obs_end', 'exp_time', 'dark_time', 'visit_gap', 'img_type', 'science_program', 'observation_reason', 'target_name']
visits[cols].head()

,seq_num,physical_filter,s_ra,s_dec,altitude,azimuth,sky_rotation,obs_start,obs_end,exp_time,dark_time,visit_gap,img_type,science_program,observation_reason,target_name
visit_id,,,,,,,,,,,,,,,,
2025022500170,170,SDSSr_65mm~empty,116.592093,-33.425352,15.025676,106.953613,202.678681,2025-02-26T00:57:10.860000,2025-02-26T00:57:16.085000,5.0,5.224695,37.428675,ACQ,BLOCK-305,FINAL_INFOCUS,HD 63214
2025022500171,171,SDSSr_65mm~empty,122.554238,-36.159018,19.792435,113.610711,359.999890,2025-02-26T00:59:30.518000,2025-02-26T01:00:00.744000,30.0,30.226869,134.432636,OBJECT,BLOCK-306,imaging,Photo08000-1
2025022500172,172,SDSSr_65mm~empty,122.560925,-36.190501,19.529781,113.862138,359.999910,2025-02-26T01:00:52.951000,2025-02-26T01:01:23.172000,30.0,30.221848,52.206098,OBJECT,BLOCK-306,imaging,Photo08000-1
2025022500173,173,SDSSr_65mm~empty,122.555987,-36.188417,19.251701,114.029109,359.999921,2025-02-26T01:02:16.061000,2025-02-26T01:02:46.305000,30.0,30.243502,52.889006,OBJECT,BLOCK-306,imaging,Photo08000-1
2025022500174,174,SDSSr_65mm~empty,122.531842,-36.228871,18.974358,114.337359,359.999960,2025-02-26T01:03:37.409000,2025-02-26T01:04:07.635000,30.0,30.226867,51.103664,OBJECT,BLOCK-306,imaging,Photo08000-1


In [291]:
imaging = visits.query('observation_reason == "imaging"')
# Does exposure time start-end still match dark time? looks like yes. 
(imaging['obs_end_mjd'] - imaging['obs_start_mjd']).values * 24 * 60 * 60 - imaging['dark_time'].values
(imaging['visit_gap'] + imaging['dark_time'])[1:].mean(), imaging['visit_gap'][1:].mean()

(np.float64(82.86123467675809), np.float64(52.63045451803399))

In [292]:
# Look like time between simple imaging visits is 52.6 seconds on average. 
# We can model this as 'readout time' for sim_runner, to ensure that exposures don't get scheduled closer than this. 
# Need to check on comparison with slew time, and if this overlaps with slew or no.
# But might depend on where this is coming from - does overslew get activate for everything? 

In [296]:
t_start = Time(imaging['obs_start_mjd'].values[0], format='mjd', scale='tai').utc
t_end = Time(imaging['obs_end_mjd'].values[-1] + (3/60/60/24), format='mjd', scale='tai').utc
topic = "lsst.sal.Script.logevent_logMessage"
fields = '*'
script = await endpoints['efd'].select_time_series(topic, fields, t_start, t_end)
topic = "lsst.sal.ATCamera.logevent_endOfImageTelemetry"
fields = ['imageName', 'imageIndex', 'exposureTime', 'darkTime', 'measuredShutterOpenTime', 
              'additionalValues', 'timestampAcquisitionStart', 'timestampDateEnd', 'timestampDateObs']
im = await endpoints['efd'].select_time_series(topic, fields, t_start, t_end)
im.rename({'imageName': 'filePath', 'additionalValues': 'functionName'}, axis=1, inplace=True)
messages = pd.concat([script, im])
messages.sort_index(inplace=True)
#display(HTML(messages.to_html()))

In [297]:
# I think this identifies timestamp for slew start, overslew start, and when the exposure is done (how accurate are timestamps?)
startslew = messages.dropna(axis=0, subset=['message']).query('message.str.contains("Slew and track")')
overslew = messages.dropna(axis=0, subset=['message']).query('message.str.contains("Overslew")')
expdone = messages.dropna(axis=0, subset=['message']).query('message.str.contains("Completed exposure")')[1:]

In [299]:
# Time for initial slew
slewtime = (overslew.index - startslew.index).values / np.timedelta64(1, 's')
print('initial slew', slewtime.mean())
# Time for overslew
overslewtime = (expdone.index - overslew.index).values / np.timedelta64(1, 's') - 30.2
print('overslew', overslewtime.mean())

initial slew 11.165386
overslew 40.941214142857135


In [300]:
list(zip((slewtime + overslewtime), imaging['visit_gap'].values[1:]))

[(np.float64(51.670355), np.float64(52.20609821844846)),
 (np.float64(52.38884399999999), np.float64(52.88900602608919)),
 (np.float64(50.57978), np.float64(51.10366379376501)),
 (np.float64(54.04518999999999), np.float64(54.566752910614014)),
 (np.float64(52.543479), np.float64(53.08568445034325)),
 (np.float64(53.422191000000005), np.float64(53.94537728279829)),
 (np.float64(50.096362), np.float64(50.616598944179714))]

In [301]:
(slewtime + overslewtime) - imaging['visit_gap'].values[1:]  # .. just to double-check if there's anything else going on, but it mostly seems like the timestamps around the slew match up with the visit_gap

array([-0.53574322, -0.50016203, -0.52388379, -0.52156291, -0.54220545,
       -0.52318628, -0.52023694])